In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import getpass

In [4]:
openai_api_key = getpass.getpass(prompt='Openai Api Key')

Openai Api Key ········


In [10]:
from langchain.tools import tool

@tool
def rule_based_support(query: str) -> str:
    """Handles predefined customer support queries using rules."""
    rules = {
        "reset password": "To reset your password, click on 'Forgot Password' on the login page.",
        "refund policy": "Our refund policy allows returns within 30 days with a receipt.",
        "cancel subscription": "To cancel, go to your account settings and click 'Cancel Subscription'."
    }
    
    for keyword, response in rules.items():
        if keyword in query.lower():
            return response

    return "I'm not sure about that. Let me transfer you to an expert."


In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=openai_api_key)

def ml_agent_response(query: str) -> str:
    """Uses GPT-4 to generate responses for complex queries."""
    messages = [
        SystemMessage(content="You are an AI support agent."),
        HumanMessage(content=query)
    ]
    return llm(messages).content


In [19]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

# Define tools for each agent
tools = [
    Tool(name="Rule-Based Support", func=rule_based_support, description="Handles predefined support queries."),
    Tool(name="ML Agent", func=ml_agent_response, description="Handles complex queries using GPT-4.")
]

# Initialize the agent
customer_support_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)


In [20]:
query_1 = "reset password"
query_2 = "How can I reset my password?"

print("Query 1:", customer_support_agent.run(query_1))  # Rule-based response
print("Query 2:", customer_support_agent.run(query_2))  # ML-based response




> Entering new AgentExecutor chain...
The question is about resetting a password, which is typically a predefined support query. 
Action: Rule-Based Support
Action Input: reset password
Observation: To reset your password, click on 'Forgot Password' on the login page.
Thought:I now know the final answer. 

Final Answer: To reset your password, click on 'Forgot Password' on the login page.

> Finished chain.
Query 1: To reset your password, click on 'Forgot Password' on the login page.


> Entering new AgentExecutor chain...
To determine the best way to answer this question, I need to consider whether it is a common support query or requires a more complex response. Resetting a password is typically a common support issue.

Action: Rule-Based Support
Action Input: "How can I reset my password?"
Observation: I'm not sure about that. Let me transfer you to an expert.
Thought:The Rule-Based Support tool was unable to provide an answer, so I should use the ML Agent to handle this query.



In [21]:
query_1 = "What are the best features of your premium plan?"
query_2 = "What are the best features of your premium plan?"

print("Query 1:", customer_support_agent.run(query_1))  # Rule-based response
print("Query 2:", customer_support_agent.run(query_2))  # ML-based response




> Entering new AgentExecutor chain...
To answer this question, I need to determine if it falls under a predefined support query or if it requires a more complex response. Since the question is about the features of a premium plan, it might be covered by predefined support queries.

Action: Rule-Based Support
Action Input: "What are the best features of your premium plan?"
Observation: I'm not sure about that. Let me transfer you to an expert.
Thought:The question about the best features of a premium plan was not covered by the predefined support queries, so I will need to use a more complex approach to provide an answer.

Action: ML Agent
Action Input: "What are the best features of your premium plan?"
Observation: To provide you with the best information about a premium plan, I would need to know which service or product you are referring to. Premium plans can vary significantly across different platforms and services. Generally, premium plans might offer features such as:

1. **Enh